## 最終課題

In [ ]:
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse
import time
from collections import deque

# スタートURL（トップページ）
start_url = "https://www.musashino-u.ac.jp/"
domain = urlparse(start_url).netloc

# User-Agent設定（botとみなされないように）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0.5993.70 Safari/537.36"
}

# 結果を格納する辞書
site_dict = {}

# BFS（幅優先探索）で全ページを巡回
queue = deque([start_url])
visited = set()

# 除外する拡張子（画像・PDFなど）
exclude_ext = (".jpg", ".jpeg", ".png", ".gif", ".svg", ".pdf", ".zip", ".mp4", ".doc", ".docx")

while queue:
    url = queue.popleft()

    if url in visited:
        continue
    visited.add(url)

    # ファイル系URLは除外
    if url.lower().endswith(exclude_ext):
        continue

    try:
        res = requests.get(url, headers=headers, timeout=10)
        if res.status_code != 200:
            continue
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, "html.parser")

        # コメントアウト内のリンクを除去
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # タイトルを取得
        title = soup.title.string.strip() if soup.title and soup.title.string else "No Title"
        site_dict[url] = title
        print(f"✔ {url} → {title}")

        # 全<a>タグのリンクを収集
        for a in soup.find_all("a", href=True):
            link = urljoin(url, a["href"])  # 絶対URLに変換
            parsed = urlparse(link)

            # 同一ドメインのみ & 除外拡張子でないリンク
            if parsed.netloc == domain and not link.lower().endswith(exclude_ext):
                if link not in visited:
                    queue.append(link)

        # 負荷対策（アクセス間隔）
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ Error fetching {url}: {e}")
        continue

print("\n=== 抽出完了 ===")
print(f"総ページ数：{len(site_dict)}")
print(site_dict)
